In [1]:
import fedhex as fx
from fedhex.pretrain import generation as fxgen
from fedhex.train import Checkpointer, EpochLossHistory, SelectiveProgbarLogger
from matplotlib import pyplot as plt
import numpy as np

Welcome to JupyROOT 6.30/02


2024-02-29 15:59:42.420499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# List data generation parameters
ndim = 2
sigma = 0.025
ngausx = 5
ngausy = 5
lims = ((0., 1.), (0., 1.))
seed = 0xace1ace1ace1ace1

# Specify generation 'Strategy' for this (gaussian) Generator
# Strategies for gaussian Generators specify the behavior of
# a base covariance matrix that changes over the sample space
strat = fxgen.RepeatStrategy(fxgen.DiagCov(ndim=ndim, sigma=sigma))

In [3]:
# Make a data generator instance for the specified gaussians
ggg = fx.GridGaussGenerator(cov_strat=strat, ngausx=ngausx,
                               ngausy=ngausy, seed=seed, lims=lims)
nsamp = 1000
samples, labels = ggg.generate(nsamp=nsamp)
data, cond = ggg.preproc()

In [4]:
nmade = 10
ninputs = 2
ncinputs = 2
hidden_layers = 2
hidden_units = 128
lr_tuple = (1e-3, 1e-4, 100)
mm = fx.MADEManager(nmade=nmade, ninputs=ninputs, ncinputs=ncinputs,
                    hidden_layers=hidden_layers, hidden_units=hidden_units,
                    activation="relu",
                    lr_tuple=lr_tuple)
mm.compile_model()

In [ ]:
starting_epoch = 0
end_epoch = 200
batch_size = 1024
flow_path = "./model/02-29_modelsaveload/"
loss_path = flow_path + "loss.npy"

callbacks = []

save_freq = 50 * batch_size
callbacks.append(Checkpointer(filepath=flow_path, save_freq=save_freq))

callbacks.append(EpochLossHistory(loss_path=loss_path))

log_freq = 10
callbacks.append(SelectiveProgbarLogger(1, epoch_interval=log_freq, epoch_end=end_epoch))

In [ ]:
mm.train_model(data=data, cond=cond, batch_size=batch_size,
               starting_epoch=starting_epoch, end_epoch=end_epoch,
               path=flow_path, callbacks=callbacks)

In [ ]:
ngen = 500
gen_labels_unique = [0.5, 0.5]
gen_labels = np.repeat([gen_labels_unique], ngen, axis=0)
gen_cond = ggg.norm(gen_labels, is_cond=True)

gen_data = mm.eval_model(gen_cond, ggg)

gen_samples = gen_data

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,8))
ax1: plt.Axes

ax1.scatter(gen_samples[:,0], gen_samples[:,1])
ax1.scatter(gen_labels_unique[0], gen_labels_unique[1])
ax1.set_title("Generated Data in Sample Space")
ax1.set_xlabel("X-coordinate")
ax1.set_ylabel("Y-coordinate")
ax1.set_xlim((0.4, 0.6))
ax1.set_ylim((0.4, 0.6))
pass

In [ ]:
mm.save(flow_path + "config.json")

In [ ]:
# from tensorflow import keras as tfk
# model = tfk.models.load_model(flow_path)

In [ ]:
from fedhex.train.tf._MADEflow import load_MADE

model_loaded, dist_loaded, made_list, cfg = load_MADE(flow_path=flow_path, newmodel=False)

In [ ]:
current_kwargs = {}
for i in range(len(made_list) // 2):
    current_kwargs[f"maf_{i}"] = {"conditional_input" : gen_cond}

gen_data_loaded = dist_loaded.sample(len(gen_cond), bijector_kwargs=current_kwargs)
gen_samples_loaded = ggg.denorm(gen_data, is_cond=False)

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(8,8))
ax1: plt.Axes

ax1.scatter(gen_samples_loaded[:,0], gen_samples_loaded[:,1])
ax1.scatter(gen_labels_unique[0], gen_labels_unique[1])
ax1.set_title("Generated Data in Sample Space")
ax1.set_xlabel("X-coordinate")
ax1.set_ylabel("Y-coordinate")
ax1.set_xlim((0.4, 0.6))
ax1.set_ylim((0.4, 0.6))
pass

In [ ]:
hist, xedges, yedges = np.histogram2d(gen_samples[:,0], gen_samples[:,1], range=((0.4,0.6),(0.4,0.6)))
hist_load, xedges, yedges = np.histogram2d(gen_samples_loaded[:,0], gen_samples_loaded[:,1], range=((0.4,0.6),(0.4,0.6)))

In [ ]:
from matplotlib import pyplot as plt
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True)
h1, _, _, _ = ax1.hist2d(gen_samples[:,0], gen_samples[:,1], range=((0.4,0.6),(0.4,0.6)))
h2, _, _, _ = ax2.hist2d(gen_samples_loaded[:,0], gen_samples_loaded[:,1], range=((0.4,0.6),(0.4,0.6)))
h3 = h1-h2
ax3.imshow(h3)
plt.show()